# Exercie sur dataset EEG 

1h30

Approche simple d'un dataset EEG en 4 étapes:

  * Ouvrir un fichier : signaux/triggers
  * filtrer les signaux
  * se créer un viewer

  * faire des ERP : moyenne des signaux autour des triggers (évenements) en fonction des labels du trigger




# Etape 1: lecture des datas

  * Installer un module python qui commence par **n**  et qui finit par **o**
  * Ouvrir le fichier **File_micromed_1.TRC** acquis via un système micromed.
  * Avec matplotlib afficher vite fait quelques channels sur quelques secondes + triggers
  



# Solution étape 1

# Etape 2: scpetre et filtrage des signaux

Il y a un comme d'hab du 50Hz.

Avec scipy.signal:

  * afficher les spectres
  * filtrer les siganux entre 0.5 - 49 Hz
  
Tips:
 
   * Attention scipy.signal.filtfilt plutôt que lfilter
   

# solution étape 2

# Etape 3: créer un *viewer custum*

  * avec ephyviewer se créer un viewer pour se balader dans les signaux avec:
    * les signaux
    * les signaux filtrés
    * les cartes temps fréquences
    * les triggers en liste


# Solution

# Etape 4 : moyenne des ERP par condition

Pour les channels [0,15,20,22] à partir des signaux filtrés:

  * empiler les ERP dans une fenêtre -200ms, 1000ms, pour les trois premiers triggers
  * faire la moyenne


# Mais en VRAI, utilisez Python MNE pour faire de l'EEG !

https://martinos.org/mne/stable/index.html
+
lien Neo-MNE en cours 
https://gist.github.com/agramfort/7fc27a18fcdc0e8cff3f
...